# Real-time music auto-tagging
In this tutorial, we use Essentia's TensorFlow integration to perform auto-tagging in real-time.
Additionally, this serves as an example of TensorFlow inference in streaming mode and can be easily adapted to work offline.


## Setup
To install Essentia with TensorFlow support, refer to the [Setup](https://essentia.upf.edu/tutorial_tensorflow_auto-tagging_classification_embeddings.html#setup) section of our previous *Music auto-tagging, classification, and embedding extraction* tutorial for instructions.

Additionally, we rely on the `pysoundcard` package to capture the audio loopback of the system and feed Essentia in real-time. This way we can easily test our models with any music coming from our local player or browser.

Then we import the required packages and Essentia algorithms.
In this case, we use the TensorFlow functionalities in streaming mode.


In [1]:
%matplotlib nbagg

import json

from essentia.streaming import (
    VectorInput,
    FrameCutter,
    TensorflowInputMusiCNN,
    VectorRealToTensor,
    TensorToPool,
    TensorflowPredict,
    PoolToTensor,
    TensorToVectorReal
)
from essentia import Pool, run, reset
from IPython import display
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import softmax
import soundcard as sc

# %matplotlib notebook

2023-09-28 10:51:00.806999: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2023-09-28 10:51:01.343157: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-28 10:51:01.343881: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2023-09-28 10:51:01.379969: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-28 10:51:01.380071: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 

Define the analysis parameters.
To make this demo work in real-time, we tweaked some of the analysis parameters of `MusiCNN`.
While it was trained on patches of size 187 (\~3 seconds) we set `patch_size` to 64 (\~1 second) to increase the prediction rate.
You can experiment with the `patch_size` and `display_size` parameters to modify the prediction rate to your taste.

In [2]:
with open('models/msd-musicnn-1.json', 'r') as json_file:
    metadata = json.load(json_file)

model_file = 'models/msd-musicnn-1.pb'
input_layer = metadata['schema']['inputs'][0]['name']
output_layer = metadata['schema']['outputs'][0]['name']
classes = metadata['classes']
n_classes = len(classes)

# Analysis parameters.
sample_rate = 16000
frame_size = 512 
hop_size = 256
n_bands = 96
patch_size = 64
display_size = 10

buffer_size = patch_size * hop_size

Instantiate the algorithms. With this, we create a network similar to the one used inside `TensorflowPredictMusiCNN`, the wrapper algorithm presented in the previous tutorial. However, by instantiating the algorithms separately we gain additional control required for real-time usage.

In [3]:
buffer = np.zeros(buffer_size, dtype='float32')
vimp = VectorInput(buffer)
fc = FrameCutter(frameSize=frame_size, hopSize=hop_size)
tim = TensorflowInputMusiCNN()
vtt = VectorRealToTensor(shape=[1, 1, patch_size, n_bands],
                         lastPatchMode='discard')
ttp = TensorToPool(namespace=input_layer)
tfp = TensorflowPredict(graphFilename=model_file,
                        inputs=[input_layer],
                        outputs=[output_layer])
ptt = PoolToTensor(namespace=output_layer)
ttv = TensorToVectorReal()
pool = Pool()

2023-09-28 10:33:52.623472: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-28 10:33:52.623605: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1660 Ti computeCapability: 7.5
coreClock: 1.59GHz coreCount: 24 deviceMemorySize: 5.80GiB deviceMemoryBandwidth: 268.26GiB/s
2023-09-28 10:33:52.623706: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-28 10:33:52.623800: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-28 10:33:52.623858: I tensorflow/core/com

Connect the algorithms. We also store the mel-spectrograms in the `Pool` for visualization purposes.

In [4]:
vimp.data   >> fc.signal
fc.frame    >> tim.frame
tim.bands   >> vtt.frame
tim.bands   >> (pool, 'melbands')
vtt.tensor  >> ttp.tensor
ttp.pool    >> tfp.poolIn
tfp.poolOut >> ptt.pool
ptt.tensor  >> ttv.tensor
ttv.frame   >> (pool, output_layer)

Create a callback function that will be called every time the audio buffer is ready to process.

In [5]:
def callback(data):
    buffer[:] = data.flatten()

    # Generate predictions.
    reset(vimp)
    run(vimp)

    # Update the mel-spectrograms and activations buffers.
    mel_buffer[:] = np.roll(mel_buffer, -patch_size)
    mel_buffer[:, -patch_size:] = pool['melbands'][-patch_size:, :].T
    img_mel.set_data(mel_buffer)
    
    act_buffer[:] = np.roll(act_buffer, -1)
    act_buffer[:, -1] = softmax(20 * pool[output_layer][-1, :].T)
    img_act.set_data(act_buffer)

    f.canvas.draw()

Initialize the plots and start processing the loopback stream.

In [ ]:
mel_buffer = np.zeros([n_bands, patch_size * display_size])
act_buffer = np.zeros([n_classes, display_size])

pool.clear()

f, ax = plt.subplots(1, 2, figsize=[9.6, 7])
f.canvas.draw()

ax[0].set_title('Mel Spectrogram')
img_mel = ax[0].imshow(mel_buffer, aspect='auto',
                       origin='lower', vmin=0, vmax=6)
ax[0].set_xticks([])

ax[1].set_title('Activations')
img_act = ax[1].matshow(act_buffer, aspect='0.5', vmin=0, vmax=1)
ax[1].set_xticks([])
ax[1].yaxis.set_ticks_position('right')
plt.yticks(np.arange(n_classes), classes, fontsize=6)

# Capture and process the speakers loopback.
with sc.all_microphones(include_loopback=True)[0].recorder(samplerate=sample_rate) as mic:
    while True:
        callback(mic.record(numframes=buffer_size).mean(axis=1))

2023-09-28 10:34:03.713105: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-28 10:34:03.713541: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1660 Ti computeCapability: 7.5
coreClock: 1.59GHz coreCount: 24 deviceMemorySize: 5.80GiB deviceMemoryBandwidth: 268.26GiB/s
2023-09-28 10:34:03.713687: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-28 10:34:03.714082: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-28 10:34:03.714371: I tensorflow/core/com

In [ ]:
# %matplotlib nbagg
# %matplotlib notebook
import json

from essentia.streaming import (
    VectorInput,
    FrameCutter,
    TensorflowInputMusiCNN,
    VectorRealToTensor,
    TensorToPool,
    TensorflowPredict,
    PoolToTensor,
    TensorToVectorReal
)
from essentia import Pool, run, reset
from IPython import display
import numpy as np
import matplotlib.pyplot as plt
# import matplotlib
from scipy.special import softmax
import soundcard as sc


with open('models/msd-musicnn-1.json', 'r') as json_file:
    metadata = json.load(json_file)

model_file = 'models/msd-musicnn-1.pb'
input_layer = metadata['schema']['inputs'][0]['name']
output_layer = metadata['schema']['outputs'][0]['name']
classes = metadata['classes']
n_classes = len(classes)

# Analysis parameters.
sample_rate = 16000
frame_size = 512 
hop_size = 256
n_bands = 96
patch_size = 64
display_size = 10

buffer_size = patch_size * hop_size

buffer = np.zeros(buffer_size, dtype='float32')
vimp = VectorInput(buffer)
fc = FrameCutter(frameSize=frame_size, hopSize=hop_size)
tim = TensorflowInputMusiCNN()
vtt = VectorRealToTensor(shape=[1, 1, patch_size, n_bands],
                         lastPatchMode='discard')
ttp = TensorToPool(namespace=input_layer)
tfp = TensorflowPredict(graphFilename=model_file,
                        inputs=[input_layer],
                        outputs=[output_layer])
ptt = PoolToTensor(namespace=output_layer)
ttv = TensorToVectorReal()
pool = Pool()


vimp.data   >> fc.signal
fc.frame    >> tim.frame
tim.bands   >> vtt.frame
tim.bands   >> (pool, 'melbands')
vtt.tensor  >> ttp.tensor
ttp.pool    >> tfp.poolIn
tfp.poolOut >> ptt.pool
ptt.tensor  >> ttv.tensor
ttv.frame   >> (pool, output_layer)


# def callback(data):
#     buffer[:] = data.flatten()

#     # Generate predictions.
#     reset(vimp)
#     run(vimp)

#     # Update the mel-spectrograms and activations buffers.
#     mel_buffer[:] = np.roll(mel_buffer, -patch_size)
#     mel_buffer[:, -patch_size:] = pool['melbands'][-patch_size:, :].T
#     img_mel.set_data(mel_buffer)
    
#     act_buffer[:] = np.roll(act_buffer, -1)
#     act_buffer[:, -1] = softmax(20 * pool[output_layer][-1, :].T)
#     img_act.set_data(act_buffer)
    
#     f.canvas.draw()


def callback(data):
    buffer[:] = data.flatten()

    # Generate predictions.
    reset(vimp)
    run(vimp)

    # Update the mel-spectrograms and activations buffers.
    mel_buffer[:] = np.roll(mel_buffer, -patch_size)
    mel_buffer[:, -patch_size:] = pool['melbands'][-patch_size:, :].T

    # Print mel_buffer
    print("Mel Spectrogram Buffer:")
    print(mel_buffer)

    act_buffer[:] = np.roll(act_buffer, -1)
    act_buffer[:, -1] = softmax(20 * pool[output_layer][-1, :].T)

    # Print act_buffer
    print("Activation Buffer:")
    print(act_buffer)



mel_buffer = np.zeros([n_bands, patch_size * display_size])
act_buffer = np.zeros([n_classes, display_size])

pool.clear()

f, ax = plt.subplots(1, 2, figsize=[9.6, 7])
f.canvas.draw()

ax[0].set_title('Mel Spectrogram')
img_mel = ax[0].imshow(mel_buffer, aspect='auto',
                       origin='lower', vmin=0, vmax=6)
ax[0].set_xticks([])

ax[1].set_title('Activations')
img_act = ax[1].matshow(act_buffer, aspect='0.5', vmin=0, vmax=1)
ax[1].set_xticks([])
ax[1].yaxis.set_ticks_position('right')
plt.yticks(np.arange(n_classes), classes, fontsize=6)

# Capture and process the speakers loopback.
with sc.all_microphones(include_loopback=True)[0].recorder(samplerate=sample_rate) as mic:
    while True:
        callback(mic.record(numframes=buffer_size).mean(axis=1))

2023-09-28 11:03:56.552972: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2023-09-28 11:03:57.150977: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-28 11:03:57.151826: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2023-09-28 11:03:57.200306: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-28 11:03:57.200753: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 

Mel Spectrogram Buffer:
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Activation Buffer:
[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.11007678]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.01541041]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.0335261 ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.06366199]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.17708507]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.01520404]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.01212032]
 [0.         0.         0.         0.         0.    

2023-09-28 11:04:03.039242: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-28 11:04:03.039700: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1660 Ti computeCapability: 7.5
coreClock: 1.59GHz coreCount: 24 deviceMemorySize: 5.80GiB deviceMemoryBandwidth: 268.26GiB/s
2023-09-28 11:04:03.039933: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-28 11:04:03.040343: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-28 11:04:03.040637: I tensorflow/core/com

Mel Spectrogram Buffer:
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Activation Buffer:
[[0.         0.         0.         0.         0.         0.
  0.         0.11007678 0.11007678 0.11007678]
 [0.         0.         0.         0.         0.         0.
  0.         0.01541041 0.01541041 0.01541041]
 [0.         0.         0.         0.         0.         0.
  0.         0.0335261  0.0335261  0.0335261 ]
 [0.         0.         0.         0.         0.         0.
  0.         0.06366199 0.06366199 0.06366199]
 [0.         0.         0.         0.         0.         0.
  0.         0.17708507 0.17708507 0.17708507]
 [0.         0.         0.         0.         0.         0.
  0.         0.01520404 0.01520404 0.01520404]
 [0.         0.         0.         0.         0.         0.
  0.         0.01212032 0.01212032 0.01212032]
 [0.         0.         0.         0.         0.    

2023-09-28 11:04:04.063293: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-28 11:04:04.063395: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1660 Ti computeCapability: 7.5
coreClock: 1.59GHz coreCount: 24 deviceMemorySize: 5.80GiB deviceMemoryBandwidth: 268.26GiB/s
2023-09-28 11:04:04.063428: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-28 11:04:04.063503: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-28 11:04:04.063560: I tensorflow/core/com

...

Mel Spectrogram Buffer:
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Activation Buffer:
[[0.11007678 0.11007678 0.11007678 0.11007678 0.11007678 0.11007678
  0.11007678 0.11007678 0.11007678 0.11007678]
 [0.01541041 0.01541041 0.01541041 0.01541041 0.01541041 0.01541041
  0.01541041 0.01541041 0.01541041 0.01541041]
 [0.0335261  0.0335261  0.0335261  0.0335261  0.0335261  0.0335261
  0.0335261  0.0335261  0.0335261  0.0335261 ]
 [0.06366199 0.06366199 0.06366199 0.06366199 0.06366199 0.06366199
  0.06366199 0.06366199 0.06366199 0.06366199]
 [0.17708507 0.17708507 0.17708507 0.17708507 0.17708507 0.17708507
  0.17708507 0.17708507 0.17708507 0.17708507]
 [0.01520404 0.01520404 0.01520404 0.01520404 0.01520404 0.01520404
  0.01520404 0.01520404 0.01520404 0.01520404]
 [0.01212032 0.01212032 0.01212032 0.01212032 0.01212032 0.01212032
  0.01212032 0.01212032 0.01212032 0.0121203

2023-09-28 11:06:33.569168: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-28 11:06:33.569305: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1660 Ti computeCapability: 7.5
coreClock: 1.59GHz coreCount: 24 deviceMemorySize: 5.80GiB deviceMemoryBandwidth: 268.26GiB/s
2023-09-28 11:06:33.569377: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-28 11:06:33.569451: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-28 11:06:33.569505: I tensorflow/core/com

Mel Spectrogram Buffer:
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Activation Buffer:
[[0.11007678 0.11007678 0.11007678 0.11007678 0.11007678 0.11007678
  0.11007678 0.11007678 0.11007678 0.11007678]
 [0.01541041 0.01541041 0.01541041 0.01541041 0.01541041 0.01541041
  0.01541041 0.01541041 0.01541041 0.01541041]
 [0.0335261  0.0335261  0.0335261  0.0335261  0.0335261  0.0335261
  0.0335261  0.0335261  0.0335261  0.0335261 ]
 [0.06366199 0.06366199 0.06366199 0.06366199 0.06366199 0.06366199
  0.06366199 0.06366199 0.06366199 0.06366199]
 [0.17708507 0.17708507 0.17708507 0.17708507 0.17708507 0.17708507
  0.17708507 0.17708507 0.17708507 0.17708507]
 [0.01520404 0.01520404 0.01520404 0.01520404 0.01520404 0.01520404
  0.01520404 0.01520404 0.01520404 0.01520404]
 [0.01212032 0.01212032 0.01212032 0.01212032 0.01212032 0.01212032
  0.01212032 0.01212032 0.01212032 0.0121203

2023-09-28 11:06:34.588638: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-28 11:06:34.588778: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1660 Ti computeCapability: 7.5
coreClock: 1.59GHz coreCount: 24 deviceMemorySize: 5.80GiB deviceMemoryBandwidth: 268.26GiB/s
2023-09-28 11:06:34.588847: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-28 11:06:34.588926: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-28 11:06:34.588983: I tensorflow/core/com

Mel Spectrogram Buffer:
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Activation Buffer:
[[0.11007678 0.11007678 0.11007678 0.11007678 0.11007678 0.11007678
  0.11007678 0.11007678 0.11007678 0.11007678]
 [0.01541041 0.01541041 0.01541041 0.01541041 0.01541041 0.01541041
  0.01541041 0.01541041 0.01541041 0.01541041]
 [0.0335261  0.0335261  0.0335261  0.0335261  0.0335261  0.0335261
  0.0335261  0.0335261  0.0335261  0.0335261 ]
 [0.06366199 0.06366199 0.06366199 0.06366199 0.06366199 0.06366199
  0.06366199 0.06366199 0.06366199 0.06366199]
 [0.17708507 0.17708507 0.17708507 0.17708507 0.17708507 0.17708507
  0.17708507 0.17708507 0.17708507 0.17708507]
 [0.01520404 0.01520404 0.01520404 0.01520404 0.01520404 0.01520404
  0.01520404 0.01520404 0.01520404 0.01520404]
 [0.01212032 0.01212032 0.01212032 0.01212032 0.01212032 0.01212032
  0.01212032 0.01212032 0.01212032 0.0121203